# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import pandas
import numpy as np
from azureml.core import Workspace, Dataset, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import joblib
import os
import logging
from azureml.widgets import RunDetails
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
import requests

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'hyptuniot'

experiment=Experiment(ws, experiment_name)

In [ ]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required


subscription_id = '6597c678-9946-48a5-a6fe-e27536a47cc0'
resource_group = 'aimazin1'
workspace_name = 'wsi'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='gas')
dataset.to_pandas_dataframe()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_amount = 0.2,
                      evaluation_interval=1,
                      delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        # There's only one parameter, so grid sampling will try each value - with multiple parameters it would try every combination
        '--C': uniform( 0.01,1.0),
        '--max_iter': choice(range(95,105))
    }
)

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=os.getcwd()+'/training',
              entry_script='train.py',
              inputs=[dataset.as_named_input('gas')], # Pass the dataset as an input...
              compute_target = compute_target)
              

hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                                      hyperparameter_sampling=param_sampling, 
                                      policy=early_termination_policy, 
                                      primary_metric_name='Accuracy', 
                                      primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                      max_total_runs=25,
                                      max_concurrent_runs=4)

In [ ]:
#TODO: Submit your experiment
hyprun = exp.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyprun).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

model = hyprun.get_best_run_by_primary_metric()
model = model.register_model(model_name='hyptun_iot', model_path='outputs/hyptun_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:

# Load image data
data = [1.25, 1.5]
# Post raw data to scoring URI
res = request.post(url=service[0].scoring_uri, data=data, headers={'R7': np.float()})

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.scoring_uri)
print(service.swagger_uri)
service.delete()